In [5]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [12]:
# But since you’re in a notebook:
root_dir = os.getcwd()

# Build the path dynamically
file_path = os.path.join(root_dir, 'data', 'spam.tsv')
if not os.path.exists(file_path):
    file_path = os.path.join(os.path.dirname(root_dir), 'data', 'spam.tsv')

df = pd.read_csv(file_path, sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
